In [1]:
#installing pdfquery, library that helps extract data from PDF files
!pip install pdfquery

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pdfquery: filename=pdfquery-0.4.3-py3-none-any.whl size=16844 sha256=b3c3f4a0687d39978547bd886765284202c6a36785c6b05cadf9b1f5a46a443a
  Stored in directory: c:\users\adity\appdata\local\pip\cache\wheels\cd\a4\8e\0a60850f5ecf4ebd3d78f902b7ee8a4c714b4d4bebefcdb859
Successfully built pdfquery
  Attempting uninstall: cssselect
    Found existing installation: cssselect 1.1.0
    Uninstalling cssselect-1.1.0:
      Successfully uninstalled cssselect-1.1.0


In [11]:
#importing all the neccessary libraries
import pdfquery                     
from collections import Counter     
import math
import requests
import json
import re

In [12]:
#defining a funciton to calculate the cosine similarity between the query entered by used and relevant document from the corpus
def cosine_similarity(query, document):
    #tokenization
    query_tokens = query.lower().split(" ")
    document_tokens = document.lower().split(" ")
    
    #counter for query and document
    query_counter = Counter(query_tokens)
    document_counter = Counter(document_tokens)
    
    #calculating dot prod
    dot_product = sum(query_counter[token] * document_counter[token] for token in query_counter.keys())
    
    #calculating magnitudes
    query_magnitude = math.sqrt(sum(query_counter[token]**2 for token in query_counter))
    document_magnitude = math.sqrt(sum(document_counter[token]**2 for token in document_counter))
    
    #calculating similarity
    #incase (query_magnitude*document_magnitude)=0, the value of similarity becomes 0 else as calculated
    similarity = dot_product/(query_magnitude*document_magnitude) if query_magnitude*document_magnitude!=0 else 0
    
    return similarity

In [13]:
#function to return a document similar to the query from the corpus
def return_response(query, corpus):
    similarities = []
    for doc in corpus:
        similarity = cosine_similarity(query, doc)      #calls the cosine_similarity function
        similarities.append(similarity)                 #adds the calculated similarity score to the similarities list
        
    return corpus[similarities.index(max(similarities))]

In [14]:
# Creating an infinite loop until the user wants to terminate the program
while True:
    print("Choose the RAG model you want to interact with: ")
    print("1. Diseases and its symptoms")
    print("2. Diseases and its treatments")
    print("3. Terminate the program")
    
    choice = int(input("Enter your choice: "))  # Accepting the model choice of the user from the menu displayed

    pdf = []
    if choice == 1:
        pdf = pdfquery.PDFQuery(r"C:\Users\adity\OneDrive\Desktop\DISEASES AND ITS SYMPTOMS.pdf")
    elif choice == 2:
        pdf = pdfquery.PDFQuery(r"C:\Users\adity\OneDrive\Desktop\DISEASES AND ITS TREATEMENTS.pdf")
    else:
        print("Closing Program.. Thank you!!")
        break

    pdf.load()
    
    # Extract all text elements
    text_elements = pdf.pq('LTTextLineHorizontal')
    text = " ".join([t.text for t in text_elements])  # Combine all text
    
    # Split text into sentences using regex
    corpus = re.split(r'(?<=[.!?]) +', text)
    
    while True:
        print("Enter Quit to exit from the model")
        user_query = input(">>> ")

        relevant_document = return_response(user_query, corpus)

        if user_query.lower() == 'quit':
            print("\n\n")
            print(f"{'=' * 30}")
            break

        full_response = []

        prompt = """
        You are a medical assistant bot that helps patients identify potential diseases based on symptoms and provides relevant symptoms for specified diseases.  
        You answer only in 60 words, ensuring you encourage the patient to follow the treatment.  
        These are the symptoms/disease provided: {user_input}  
        Provide the most appropriate disease or symptoms while reassuring the patient and encouraging them to seek medical care and follow treatment.  
        """

        url = "http://localhost:11434/api/generate"
    
        # Now requesting the above URL which is running locally to execute the below command
        # We are using JSON format for making the request

        data = {
            "model": "llama3",
            "prompt": prompt.format(user_input=user_query, relevant_document=relevant_document)
        }

        headers = {'Content-Type': 'application/json'}

        response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)

        try:
            for line in response.iter_lines():
                # Filter out keep-alive new lines
                if line:
                    decoded_line = json.loads(line.decode('utf-8'))
                    full_response.append(decoded_line['response'])
        finally:
            response.close()

        print(''.join(full_response))
        print(f"{'*' * 30}")


Choose the RAG model you want to interact with: 
1. Diseases and its symptoms
2. Diseases and its treatments
3. Terminate the program
Enter your choice: 1
Enter Quit to exit from the model
>>> what is diabetes
Diabetes is a chronic condition characterized by high blood sugar levels. Common symptoms include increased thirst and urination, fatigue, blurred vision, and recurring skin infections. If you're experiencing these symptoms, don't hesitate! See your doctor for proper diagnosis and treatment. With proper management, you can control your diabetes and enjoy a healthy life.
******************************
Enter Quit to exit from the model
>>> 2
I'm here to help! Please provide the two symptoms, and I'll do my best to identify a potential disease or provide relevant information. Remember, it's always important to consult with a healthcare professional for an accurate diagnosis and proper treatment.

Please share the two symptoms you're experiencing.
******************************
Enter